## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

In [156]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
import asyncio
from dotenv import load_dotenv
from openai import OpenAI, AsyncOpenAI
from anthropic import AsyncAnthropic
from pydantic import BaseModel

In [157]:
# Always remember to do this!
load_dotenv(override=True)

True

In [158]:
# Print the key prefixes to help with any debugging

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

if OPENAI_API_KEY:
    print(f"OpenAI API Key exists and begins {OPENAI_API_KEY[:8]}")
else:
    print("OpenAI API Key not set")
    
if ANTHROPIC_API_KEY:
    print(f"Anthropic API Key exists and begins {ANTHROPIC_API_KEY[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if GOOGLE_API_KEY:
    print(f"Google API Key exists and begins {GOOGLE_API_KEY[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if DEEPSEEK_API_KEY:
    print(f"DeepSeek API Key exists and begins {DEEPSEEK_API_KEY[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if GROQ_API_KEY:
    print(f"Groq API Key exists and begins {GROQ_API_KEY[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (and this is optional)
Google API Key exists and begins AI
DeepSeek API Key not set (and this is optional)
Groq API Key not set (and this is optional)


In [159]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [160]:
print(messages)

[{'role': 'user', 'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]


In [161]:
openai = AsyncOpenAI()
response = await openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)


What are the ethical implications of developing highly autonomous artificial intelligence systems in terms of accountability, decision-making, and the potential impact on societal norms?


In [162]:
# Define Pydantic model for storing LLM results
class LLMResult(BaseModel):
    model: str
    answer: str


In [163]:
results: list[LLMResult] = []
messages = [{"role": "user", "content": question}]

In [182]:
# The API we know well
async def openai_answer() -> None:

  if OPENAI_API_KEY is None:
    return None
  
  print("OpenAI starting!")
  model_name = "gpt-4o-mini"

  try:
    response = await openai.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content
    results.append(LLMResult(model=model_name, answer=answer))
  except Exception as e:
    print(f"Error with OpenAI: {e}")
    return None

  print("OpenAI done!")

In [165]:
# Anthropic has a slightly different API, and Max Tokens is required

async def anthropic_answer() -> None:

  if ANTHROPIC_API_KEY is None:
    return None
  
  print("Anthropic starting!")
  model_name = "claude-3-7-sonnet-latest"

  claude = AsyncAnthropic()
  try:
    response = await claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
    answer = response.content[0].text
    results.append(LLMResult(model=model_name, answer=answer))
  except Exception as e:
    print(f"Error with Anthropic: {e}")
    return None

  print("Anthropic done!")

In [166]:
async def google_answer() -> None:

  if GOOGLE_API_KEY is None:
    return None
  
  print("Google starting!")
  model_name = "gemini-2.0-flash"

  gemini = AsyncOpenAI(api_key=GOOGLE_API_KEY, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
  try:
    response = await gemini.chat.completions.create(model=model_name, messages=messages)
    answer =  response.choices[0].message.content
    results.append(LLMResult(model=model_name, answer=answer))
  except Exception as e:
    print(f"Error with Google: {e}")
    return None

  print("Google done!")

In [167]:
async def deepseek_answer() -> None:

  if DEEPSEEK_API_KEY is None:
    return None
  
  print("DeepSeek starting!")
  model_name = "deepseek-chat"

  deepseek = AsyncOpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com/v1")
  try:
    response = await deepseek.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content
    results.append(LLMResult(model=model_name, answer=answer))
  except Exception as e:
    print(f"Error with DeepSeek: {e}")
    return None

  print("DeepSeek done!")

In [168]:
async def groq_answer() -> None:

  if GROQ_API_KEY is None:
    return None
  
  print("Groq starting!")
  model_name = "llama-3.3-70b-versatile"

  groq = AsyncOpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")
  try:
    response = await groq.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content
    results.append(LLMResult(model=model_name, answer=answer))
  except Exception as e:
    print(f"Error with Groq: {e}")
    return None

  print("Groq done!")


## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [169]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [170]:
async def ollama_answer() -> None:
  model_name = "llama3.2"

  print("Ollama starting!")
  ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
  try:
    response = ollama.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content
    results.append(LLMResult(model=model_name, answer=answer))
  except Exception as e:
    print(f"Error with Ollama: {e}")
    return None

  print("Ollama done!") 

In [171]:
async def gather_answers():
  tasks = [
    openai_answer(),
    anthropic_answer(),
    google_answer(),
    deepseek_answer(),
    groq_answer(),
    ollama_answer()
  ]
  await asyncio.gather(*tasks)

await gather_answers()

OpenAI starting!
Google starting!
Ollama starting!


Ollama done!
Google done!
OpenAI done!


In [181]:
together = ""
competitors = []
answers = []

for res in results:
    competitor = res.model
    answer = res.answer
    competitors.append(competitor)
    answers.append(answer)
    together += f"# Response from competitor {competitor}\n\n"
    together += answer + "\n\n"

print(f"Number of competitors: {len(results)}")
print(together)


Number of competitors: 3
# Response from competitor llama3.2

The development of highly autonomous artificial intelligence (AI) systems raises several ethical implications related to accountability, decision-making, and societal norms. Some of the key concerns include:

1. **Accountability**: As AI systems become more autonomous, it becomes increasingly challenging to assign responsibility for their actions. Who is accountable for an AI's decisions? The developer, the end-user, or the AI itself? This lack of clear accountability raises concerns about the potential for harm caused by faulty or biased AI systems.
2. **Decision-making**: Autonomous AI systems will be making decisions without human oversight, which can lead to unknown and unintended consequences. Decision-making algorithms may prioritize efficiency or profit over human values, potentially leading to harmful outcomes.
3. **Societal norms**: The introduction of autonomous AI systems can disrupt societal norms and values, par

In [174]:
judge = f"""You are judging a competition between {len(results)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [175]:
print(judge)

You are judging a competition between 3 competitors.
Each model has been given this question:

What are the ethical implications of developing highly autonomous artificial intelligence systems in terms of accountability, decision-making, and the potential impact on societal norms?

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}

Here are the responses from each competitor:

# Response from competitor llama3.2

The development of highly autonomous artificial intelligence (AI) systems raises several ethical implications related to accountability, decision-making, and societal norms. Some of the key concerns include:

1. **Accountability**: As AI systems become more autonomous, it becomes increasingly challenging to assign responsibility for their acti

In [176]:
judge_messages = [{"role": "user", "content": judge}]

In [177]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
judgement = response.choices[0].message.content
print(judgement)


{"results": ["gemini-2.0-flash", "gpt-4o-mini", "llama3.2"]}


In [178]:
# OK let's turn this into results!

results_dict = json.loads(judgement)
ranks = results_dict["results"]
for index, comp in enumerate(ranks):
    print(f"Rank {index+1}: {comp}")

Rank 1: gemini-2.0-flash
Rank 2: gpt-4o-mini
Rank 3: llama3.2
